# 02 — Supermarket Scraping & Data Joining

**Objective:** Scrape product catalogues from 3+ European supermarkets and join to Open Food Facts.

## Scrapers
1. **Mercadona** (Spain): REST API, ~8-10K products
2. **Albert Heijn** (Netherlands): Mobile API, ~24K products with EAN + nutrition
3. **Carrefour** (France/Spain): Web scraping with JS rendering

## Join Strategy
- EAN barcode match (AH) → high confidence
- Fuzzy name+brand+weight match (Mercadona) → medium confidence
- Document match rates and unmatched products in both directions

## Private Label Identification
- Mercadona: Hacendado, Deliplus, Bosque Verde, etc.
- AH: brand starts with "AH"
- Carrefour: Carrefour, Simpl, Carrefour Bio, etc.

In [ ]:
from src.data.scrapers.mercadona import MercadonaScraper
from src.data.scrapers.albert_heijn import AlbertHeijnScraper
from src.data.join import join_on_ean, join_on_fuzzy_name

In [ ]:
# TODO: Run scrapers and join to OFF